# Bilinear Maps

**Module 07** | 07-pairings

*e: G1 x G2 -> GT, bilinearity property*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **bilinear maps**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Basic familiarity with Python syntax.
- A working SageMath installation (or access to CoCalc/SageMathCell).

## What Is a Bilinear Map?

In [ ]:
# A pairing e: G1 x G2 -> GT satisfying
# e(aP, bQ) = e(P, Q)^(ab)
# TODO: set up pairing-friendly curve in SageMath

## The Bilinearity Property

In [ ]:
# Verify: e(P+P', Q) = e(P,Q) * e(P',Q)
# TODO: demonstrate bilinearity with concrete examples

## Non-Degeneracy

In [ ]:
# If P != O and Q != O, then e(P, Q) != 1
# TODO: check non-degeneracy condition

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **bilinear maps**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [The Weil Pairing](07b-weil-pairing-intuition.ipynb)